In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import sqlite3
import pickle
pd.set_option('display.max_columns', None)
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [2]:
df_data2 = pd.read_pickle("df_data2.pickle") 

In [3]:
df_data2

,District name,district code,Geo code police,Force Name,year,population,headcount,Month,LSOA name,LSOA code,amount_of_crimes,month
0,Adur,E07000223,E23000033,Sussex,2011,61334,3181,2011-01,Brighton and Hove 026A,E01016849,19,1
1,Adur,E07000223,E23000033,Sussex,2011,61334,3181,2011-01,Brighton and Hove 029A,E01016850,20,1
2,Adur,E07000223,E23000033,Sussex,2011,61334,3181,2011-01,Brighton and Hove 029B,E01016851,4,1
3,Adur,E07000223,E23000033,Sussex,2011,61334,3181,2011-01,Brighton and Hove 026B,E01016852,19,1
4,Adur,E07000223,E23000033,Sussex,2011,61334,3181,2011-01,Brighton and Hove 026C,E01016853,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4358683,Wiltshire,E06000054,E23000038,Wiltshire,2019,61334,3181,2019-12,Swindon 028E,E01032710,28,12
4358684,Wiltshire,E06000054,E23000038,Wiltshire,2019,61334,3181,2019-12,Swindon 027D,E01032711,3,12
4358685,Wiltshire,E06000054,E23000038,Wiltshire,2019,61334,3181,2019-12,Swindon 027E,E01032712,5,12
4358686,Wiltshire,E06000054,E23000038,Wiltshire,2019,61334,3181,2019-12,Swindon 019F,E01032715,27,12


In [4]:
def evaluate(model, test_features, test_labels, X_test, y_test):
    pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, pred))
    abso = mean_absolute_error(y_test, pred)
    r2 = r2_score(y_test, pred)
    print(f'The rmse of the model is {rmse}.')
    print(f'The mean abs error of the model is {abso}.')
    print(f'The r2 of the model is {r2}, using weighted average.')
    
    return r2

### https://towardsdatascience.com/how-data-normalization-affects-your-random-forest-algorithm-fbc6753b4ddf

# Headcount and population scaled (only month)

In [8]:
df_data = df_data2.copy()

In [9]:
lab = LabelEncoder()
labels_force = df_data['Force Name'].unique()
lab.fit(labels_force)
df_data[['Force Name']] = df_data[['Force Name']].apply(lab.fit_transform)

In [10]:
labels_lsoa = df_data['LSOA code'].unique()
lab.fit(labels_lsoa)
df_data[['LSOA code']] = df_data[['LSOA code']].apply(lab.fit_transform)

In [11]:
scaler = MinMaxScaler()
scaler.fit(df_data[['population']])
df_data['population'] = scaler.transform(df_data[['population']])

In [12]:
scaler.fit(df_data[['headcount']])
df_data['headcount'] = scaler.transform(df_data[['headcount']])

In [13]:
df_train = df_data[df_data['year']<2019]
df_test = df_data[df_data['year']==2019]

In [14]:
X_train1 = df_train[['Force Name', 'population', 'headcount', 'LSOA code', 'month']]
y_train1 = df_train['amount_of_crimes']

X_test1 = df_test[['Force Name', 'population', 'headcount', 'LSOA code', 'month']]
y_test1 = df_test['amount_of_crimes']

In [15]:
rf1 = RandomForestRegressor(max_depth = 10, n_jobs = -1)

In [18]:
rf1.fit(X_train1, y_train1)

RandomForestRegressor(max_depth=10, n_jobs=-1)

In [26]:
base_r2_1 = evaluate(rf1, X_train1, y_train1, X_test1, y_test1)

The rmse of the model is 15.39677563074548.
The mean abs error of the model is 9.103496985718103.
The r2 of the model is 0.1999564416243068, using weighted average.


In [27]:
X_train1

,Force Name,population,headcount,LSOA code,month
0,33,0.051875,0.073722,15724,1
1,33,0.051875,0.073722,15725,1
2,33,0.051875,0.073722,15726,1
3,33,0.051875,0.073722,15727,1
4,33,0.051875,0.073722,15728,1
...,...,...,...,...,...
35973818,39,0.435263,0.006248,30698,12
35973819,39,0.435263,0.006248,30699,12
35973820,39,0.435263,0.006248,30700,12
35973821,39,0.435263,0.006248,30701,12


In [28]:
rf1.feature_importances_

array([0.02498654, 0.00823519, 0.01519756, 0.94810593, 0.00347479])

# Headcount, population, year, month scaled

In [ ]:
df_data=df_data2.copy()

In [ ]:
lab = LabelEncoder()
labels_force = df_data['Force Name'].unique()
lab.fit(labels_force)
df_data[['Force Name']] = df_data[['Force Name']].apply(lab.fit_transform)

In [ ]:
labels_lsoa = df_data['LSOA code'].unique()
lab.fit(labels_lsoa)
df_data[['LSOA code']] = df_data[['LSOA code']].apply(lab.fit_transform)

In [ ]:
scaler = MinMaxScaler()
scaler.fit(df_data[['population']])
df_data['population'] = scaler.transform(df_data[['population']])

In [ ]:
scaler.fit(df_data[['headcount']])
df_data['headcount'] = scaler.transform(df_data[['headcount']])

In [ ]:
scaler.fit(df_data[['year']])
df_data['year'] = scaler.transform(df_data[['year']])

In [ ]:
scaler.fit(df_data[['month']])
df_data['month'] = scaler.transform(df_data[['month']])

In [ ]:
df_train = df_data[df_data['year']<2019]
df_test = df_data[df_data['year']==2019]

In [ ]:
X_train2 = df_train[['Force Name', 'LSOA code', 'population', 'headcount', 'year', 'month']]
y_train2 = df_train['amount_of_crimes']

X_test2 = df_test[['Force Name', 'LSOA code', 'population', 'headcount', 'year', 'month']]
y_test2 = df_test['amount_of_crimes']

In [ ]:
rf2 =  RandomForestRegressor(max_depth = 10, n_jobs = -1)

In [ ]:
rf2.fit(X_train2, y_train2)

In [ ]:
base_r2_2 = evaluate(rf2, X_train2, y_train2, X_test2, y_test2)

In [ ]:
X_train2

In [ ]:
rf2.feature_importances_

# RF3 Headcount, population scaled
# Month (scaled) instead of month and year

In [41]:
df_data=df_data2.copy()

In [42]:
lab = LabelEncoder()
labels_force = df_data['Force Name'].unique()
lab.fit(labels_force)
df_data[['Force Name']] = df_data[['Force Name']].apply(lab.fit_transform)

In [43]:
labels_lsoa = df_data['LSOA code'].unique()
lab.fit(labels_lsoa)
df_data[['LSOA code']] = df_data[['LSOA code']].apply(lab.fit_transform)

In [44]:
labels_lsoa = df_data['Month'].unique()
lab.fit(labels_lsoa)
df_data[['Month']] = df_data[['Month']].apply(lab.fit_transform)

In [45]:
scaler.fit(df_data[['headcount']])
df_data['headcount'] = scaler.transform(df_data[['headcount']])

In [46]:
scaler = MinMaxScaler()
scaler.fit(df_data[['population']])
df_data['population'] = scaler.transform(df_data[['population']])

In [47]:
df_train = df_data[df_data['year']<2019]
df_test = df_data[df_data['year']==2019]

In [48]:
X_train3 = df_train[['population','headcount', 'LSOA code', 'Month']]
y_train3 = df_train['amount_of_crimes']

X_test3 = df_test[['population','headcount', 'LSOA code', 'Month']]
y_test3 = df_test['amount_of_crimes']

In [49]:
rf3 =  RandomForestRegressor(max_depth = 10, n_jobs = -1)

In [50]:
rf3.fit(X_train3, y_train3)

RandomForestRegressor(max_depth=10, n_jobs=-1)

In [51]:
base_r2_3 = evaluate(rf3, X_train3, y_train3, X_test3, y_test3)

The rmse of the model is 15.17098938568052.
The mean abs error of the model is 8.660490490314299.
The r2 of the model is 0.22324889516445456, using weighted average.


In [52]:
X_train3

,population,headcount,LSOA code,Month
0,0.051875,0.073722,15724,0
1,0.051875,0.073722,15725,0
2,0.051875,0.073722,15726,0
3,0.051875,0.073722,15727,0
4,0.051875,0.073722,15728,0
...,...,...,...,...
35973818,0.435263,0.006248,30698,95
35973819,0.435263,0.006248,30699,95
35973820,0.435263,0.006248,30700,95
35973821,0.435263,0.006248,30701,95


In [53]:
rf3.feature_importances_

array([4.03906220e-04, 1.98421362e-02, 9.57695387e-01, 2.20585703e-02])

# RF3 Not scaled

In [29]:
df_data = df_data2.copy()

In [30]:
lab = LabelEncoder()
labels_lsoa = df_data['LSOA code'].unique()
lab.fit(labels_lsoa)
df_data[['LSOA code']] = df_data[['LSOA code']].apply(lab.fit_transform)

In [31]:
labels_month = df_data['Month'].unique()
lab.fit(labels_lsoa)
df_data[['Month']] = df_data[['Month']].apply(lab.fit_transform)

In [32]:
df_train = df_data[df_data['year']<2019]
df_test = df_data[df_data['year']==2019]

In [35]:
X_train4 = df_train[['population','headcount', 'LSOA code', 'Month']]
y_train4 = df_train['amount_of_crimes']

X_test4 = df_test[['population','headcount', 'LSOA code', 'Month']]
y_test4 = df_test['amount_of_crimes']

In [36]:
rf4 =  RandomForestRegressor(max_depth = 10, n_jobs = -1)

In [37]:
rf4.fit(X_train4, y_train4)

RandomForestRegressor(max_depth=10, n_jobs=-1)

In [38]:
base_r2_4 = evaluate(rf4, X_train4, y_train4, X_test4, y_test4)

The rmse of the model is 15.181544353329823.
The mean abs error of the model is 8.669484513021828.
The r2 of the model is 0.22216769543653725, using weighted average.


In [39]:
X_train4

,population,headcount,LSOA code,Month
0,61334,3181,15724,0
1,61334,3181,15725,0
2,61334,3181,15726,0
3,61334,3181,15727,0
4,61334,3181,15728,0
...,...,...,...,...
35973818,498064,1021,30698,95
35973819,498064,1021,30699,95
35973820,498064,1021,30700,95
35973821,498064,1021,30701,95


In [40]:
rf4.feature_importances_

array([3.52037047e-04, 1.94331829e-02, 9.58074816e-01, 2.21399637e-02])

#  RF5 Officers per 100k ppl instead of headcount and pop

In [19]:
df_data = df_data2.copy()

In [20]:
df_data['Officers per 100000 people'] = df_data['headcount']/(df_data['population']/100000)

In [21]:
lab = LabelEncoder()
labels_lsoa = df_data['LSOA code'].unique()
lab.fit(labels_lsoa)
df_data[['LSOA code']] = df_data[['LSOA code']].apply(lab.fit_transform)

In [22]:
labels_month = df_data['Month'].unique()
lab.fit(labels_lsoa)
df_data[['Month']] = df_data[['Month']].apply(lab.fit_transform)

In [ ]:
df_train = df_data[df_data['year']<2019]
df_test = df_data[df_data['year']==2019]

In [ ]:
X_train5 = df_train[['Officers per 100000 people', 'LSOA code', 'Month']]
y_train5 = df_train['amount_of_crimes']

X_test5 = df_test[['Officers per 100000 people', 'LSOA code', 'Month']]
y_test5 = df_test['amount_of_crimes']

In [ ]:
rf5 =  RandomForestRegressor(max_depth = 10, n_jobs = -1)

In [ ]:
rf5.fit(X_train5, y_train5)

In [ ]:
base_r2_5 = evaluate(rf5, X_train5, y_train5, X_test5, y_test5)

In [ ]:
X_train5

In [ ]:
rf5.feature_importances_